<img src="Biểu đồ minh họa\Bảng trung gian của quan hệ N2N.drawio.png" />

<img src="Ảnh\DemoKDLv1\image.png"/>

> https://www.baeldung.com/jpa-many-to-many

trong bài viết này, nếu sử dụng embeded id thì khi thêm một bản ghi sẽ như thế nào

<details>

Bài viết mà bạn tham khảo trên Baeldung giải thích về quan hệ "Many-to-Many" trong JPA. Khi sử dụng `@EmbeddedId` để thay thế cho khóa chính composite (bao gồm nhiều trường), quá trình thêm bản ghi vào bảng liên kết (join table) sẽ khác so với khi bạn sử dụng một khóa chính đơn lẻ.

### Tóm tắt về `@EmbeddedId`

`@EmbeddedId` được sử dụng để chỉ định rằng khóa chính của bảng liên kết là một đối tượng (class) được nhúng vào, thay vì chỉ dùng một khóa chính đơn lẻ. Đối tượng này sẽ bao gồm các trường của khóa chính composite.

Giả sử bạn có một mối quan hệ "Many-to-Many" giữa hai thực thể `Student` và `Course`. Để biểu diễn quan hệ này, bạn sẽ có một bảng liên kết (`student_course`) với khóa chính composite.

Ví dụ:
```java
@Embeddable
public class StudentCourseId implements Serializable {
    private Long studentId;
    private Long courseId;

    // constructors, equals, hashcode methods
}

@Entity
public class StudentCourse {
    @EmbeddedId
    private StudentCourseId id;
    
    @ManyToOne
    @MapsId("studentId")
    private Student student;

    @ManyToOne
    @MapsId("courseId")
    private Course course;
}
```

### Quá trình thêm bản ghi khi sử dụng `@EmbeddedId`

Khi sử dụng `@EmbeddedId`, bạn không trực tiếp thao tác với khóa chính đơn lẻ (như `student_id`, `course_id`), mà sẽ thao tác với đối tượng khóa chính composite `StudentCourseId`. Khi bạn muốn thêm một bản ghi vào bảng liên kết, bạn cần tạo ra một đối tượng `StudentCourseId`, sau đó gán vào đối tượng `StudentCourse`.

Giả sử bạn có một sinh viên `student` và một khóa học `course`, bạn muốn lưu một bản ghi vào bảng `student_course`:

```java
Student student = ...;  // Lấy thông tin sinh viên
Course course = ...;    // Lấy thông tin khóa học

// Tạo đối tượng khóa chính composite
StudentCourseId studentCourseId = new StudentCourseId(student.getId(), course.getId());

// Tạo đối tượng bảng liên kết
StudentCourse studentCourse = new StudentCourse();
studentCourse.setId(studentCourseId);
studentCourse.setStudent(student);
studentCourse.setCourse(course);

// Lưu đối tượng vào cơ sở dữ liệu
studentCourseRepository.save(studentCourse);
```

### Tóm tắt cách hoạt động:
1. Bạn tạo một đối tượng `StudentCourseId` để đại diện cho khóa chính composite.
2. Bạn gán đối tượng này vào đối tượng `StudentCourse`, sau đó thiết lập các mối quan hệ với các đối tượng `Student` và `Course`.
3. Khi gọi `save()` trên `studentCourseRepository`, JPA sẽ sử dụng `StudentCourseId` để lưu vào bảng liên kết.

### Lưu ý:
- Mỗi bản ghi trong bảng `student_course` sẽ có một khóa chính composite từ hai trường: `student_id` và `course_id`.
- `@MapsId` được sử dụng để liên kết các trường trong `StudentCourseId` với các trường trong đối tượng `Student` và `Course`.
</details>

<details>
<summary>tóm tắt</summary>
cập nhật cả EmbeddedId và thực thể Id
</details>

trong thực tế người ta có khuyến khích tách riêng việc Fake dữ liệu ra khỏi StudentService và thành StudentFakingService không

<details>
<summary>Trả lời</summary>

Trong thực tế, việc tách riêng các chức năng như **Fake dữ liệu** ra khỏi **StudentService** và chuyển nó sang một service khác như **StudentFakingService** có thể là một quyết định tốt trong một số trường hợp, đặc biệt trong môi trường phát triển phần mềm theo hướng mô-đun và dễ bảo trì. Dưới đây là một số lý do tại sao việc tách biệt này có thể hữu ích:

### 1. **Dễ bảo trì và mở rộng**
   Khi có một service riêng biệt chuyên về fake dữ liệu, việc duy trì và phát triển các tính năng liên quan đến việc giả lập dữ liệu sẽ dễ dàng hơn. Bạn có thể thêm hoặc thay đổi cách fake dữ liệu mà không làm ảnh hưởng đến logic chính của StudentService.

### 2. **Giảm độ phức tạp (Separation of Concerns)**
   Một trong những nguyên lý thiết kế phần mềm quan trọng là **separation of concerns** (phân tách các mối quan tâm). Nếu StudentService thực hiện nhiều chức năng, chẳng hạn như quản lý sinh viên và fake dữ liệu, nó có thể trở nên khó khăn trong việc bảo trì và kiểm thử. Việc tách fake dữ liệu thành một service riêng biệt giúp đảm bảo mỗi service có một nhiệm vụ rõ ràng.

### 3. **Dễ kiểm thử (Testability)**
   Khi fake dữ liệu được tách ra thành một service riêng biệt, việc kiểm thử logic của StudentService và StudentFakingService sẽ dễ dàng hơn. Bạn có thể dễ dàng mô phỏng dữ liệu giả trong các bài kiểm thử mà không làm ảnh hưởng đến các phần khác của hệ thống.

### 4. **Tính tái sử dụng (Reusability)**
   Nếu giả lập dữ liệu không chỉ được sử dụng trong StudentService mà còn có thể cần ở các nơi khác, thì việc tách nó thành một service riêng sẽ giúp tái sử dụng mã dễ dàng hơn mà không cần phải sao chép mã nguồn.

### 5. **Dễ dàng thay đổi và mở rộng**
   Việc giả lập dữ liệu có thể thay đổi trong tương lai (ví dụ, thay đổi cách thức fake dữ liệu hoặc sử dụng một nguồn dữ liệu giả khác). Nếu mã này được giữ trong một service riêng biệt, bạn có thể thay đổi mà không ảnh hưởng đến logic của StudentService.

### Tuy nhiên, có một số yếu tố cần cân nhắc:
- **Đơn giản hay phức tạp**: Nếu việc fake dữ liệu chỉ là một phần nhỏ và đơn giản của StudentService, có thể không cần phải tách ra thành một service riêng biệt. Việc tách dịch vụ có thể tạo ra sự phức tạp không cần thiết.
- **Quy mô hệ thống**: Với hệ thống nhỏ, việc tách các service có thể không mang lại nhiều lợi ích, nhưng đối với hệ thống lớn, sự phân tách rõ ràng sẽ giúp dễ dàng duy trì và phát triển.

### Tổng kết:
Tóm lại, việc tách Fake dữ liệu thành **StudentFakingService** có thể là một chiến lược tốt trong trường hợp bạn muốn giữ cho mã nguồn sạch sẽ, dễ bảo trì, dễ kiểm thử và dễ mở rộng. Tuy nhiên, cần phải xem xét quy mô hệ thống và mức độ phức tạp của ứng dụng để quyết định liệu việc tách này có thực sự cần thiết hay không.
</details>

<details>
<summary>ý kiến cá nhân</summary>

khách hàng- đơn hàng- danh sách mặt hàng trong đơn

fake đơn hàng= 
- cần get fake khách hàng

nếu làm riêng biệt, 
- get N khách hàng random
    - fake N đơn hàng từ mỗi khách hàng trong đó
- get N đơn hàng random
    - fake N "mặt hàng trong đơn" từ mỗi đơn hàng trong đó
-> do chọn ngẫu nhiên nên có đơn hàng 0 sản phẩm

làm lồng nhau

/* xem bên dưới*/

-> phải tái sử dụng 
service "đơn hàng".createRandom() sẽ gọi service "mặt hàng trong đơn".createRandom() 
service "mặt hàng trong đơn".createRandom() sẽ gọi service "mặt hàng".getRandom()

-> faking phụ thuộc lẫn nhau (buộc chặt với nhau)
-> tách riêng faking
</details>

In [ ]:
create N "đơn hàng" cho N khách hàng : {
    listKhachhang= choose random N "khachHang";

    // create N đơn hàng cho 1 khách hàng
    for(KhachHang kh : listKhachhang){ 
        donHang= create 1 DonHang from null;

        listMatHang= choose random N "MatHang";

        listMathangduocdat= new ArrayList;

        // create N matHangDuocDat cho 1 đơn hàng
        for(MatHang mh : listMathang){
            mhdd1= create 1 MatHangDuocDat from (1 don hang, 1 mat hang)

            listMathangduocdat.add(mhdd1);
        }
    }
}

In [ ]:
create N KhachHang1 
    - mỗi KhachHang1 có N DonHang1
        - mỗi DonHang1 có N MatHangDuocDat1
            - mỗi MatHangDuocDat1 có 1 MatHang;

ngược lại
- create N matHangDuocDat
- create N donHang
- create N KhachHang
(không thành công, vì matHangDuocDat có đơn hàng là mẹ, phải fake mẹ từ trước)

In [ ]:
sai, 

chỉ quan hệ N-N mới cần lồng nhau như trên

A <>----composite------ B

A phải khởi tạo trước B

In [ ]:
kết luận

phần nào không bị gửi khóa= phải tạo trước

quan hệ N-N, 2 bảng mẹ có thể khởi tạo song song

In [ ]:
khởi tạo DB {

    khởi tạo VanPhongDaiDien{
        gọi faker_Vpdd();
        gọi "khởi tạo Cửa hàng"
        gọi "khởi tạo Khách hàng"
    }

    Khởi tạo MatHang{
        gọi faker_MatHang();
    }

    //---------------------------------------------------



    khởi tạo KHDL {
        gọi getRandomKh();
        gọi faker_Khdl();
    }
    khởi tạo KHBD{
        gọi getRandomKh();
        gọi faker_Khbd();
    }
}
//---------------------------------------------------

Khởi tạo KhachHang {
    gọi findAllVpdd();
    gọi "khởi tạo DonDatHang"
}

khởi tạo DonDatHang {
    gọi findAllKhachhang();
    gọi findRandomMathang();
    gọi faker_MatHangDuocDat();
}

//---------------------------------------------------

khởi tạo CuaHang {
    gọi findAllVpdd();
    gọi findRandomMathang();
    gọi faker_MatHangDuocLuuTru();
}


In [ ]:
createAndSaveVanPhongDaiDienFullFeature (N) {
    for(N times){
        vpdd1= createAndSave1VanPhongDaiDien() ; 
        listCuaHang = createAndSaveCuaHangFullFeature(N2, vpdd);
        listKhachHang = createAndSaveKhachHangFullFeature(N3, vpdd); 
    }
}
// 🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸
createAndSaveCuaHangFullFeature(N, vpdd){
    for(N times){
        ch1= createAndSave1CuaHang(vpdd);
        listMhdlt= createAndSaveMhdltFullFeature(N2);
    }
}

createAndSaveMhdltFullFeature(N1){
    listMh= Repo.findMatHangRandom(soLuong= N1);

    listMhdlt= null;

    for(MatHang mh1 : listMh){
        mhdlt1= createAndSave_1Mhdlt_From(mh1);

        listMhdlt.add(mhdlt1);
    }
}
// 🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
create

In [ ]:
N*createMH{}

N0*createVpdd{
    N1*createKH{
        create1Kh
        createKhdl{};
        createKhbd{};

        N2*createDdh{
            N3*createMhdd{
                //...
            }
        }
    };

    N4*createCH{
        create1CH

        N5*createMhdlt{
            // ...
        }
    };
}

In [ ]:
psvm main(){
    createNhieuMh(N0);
    createNhieuVpdd(N1);
}
//---------------------------------------
createNhieuMh(N0){
    for(N0 times){
        create1MH();
    }
}
// ---------------------------------------
createNhieuVpdd(N0){
    for(N0 times){
        vpdd1= create1Vpdd();
        createNhieuKh(from= vpdd1, N1);

        createNhieuCh(from= vpdd1, N2);
    }
}
// ------------------------------------
createNhieuKh(from vpdd1, N0){
    for(N0 times){
        kh1= create1Kh(from= vpdd1);

        createNhieuDdh(from= kh1, N1);
    }
}

createNhieuCh(from vpdd1, N0){
    for(N0 times){
        ch1= create1Ch();
        createNhieuMhdlt(from= ch1, N1);
    }
}

//-------------------------------------------

createNhieuDdh(from= kh1, N0){
    for(N0 times){
        ddh1= create1Ddh();

        createNhieuMhdd(from= ddh1, N1);
    }
}
//------------------------------------------

createNhieuMhdlt(from= ch1, N0){
    nhieuMhdlt= new ArrayList<>();

    for(N0 times){
        mh1= mh_Service.getRandom(1);
        mhdlt1= mhdlt_Faker.createFake(mh1);

        nhieuMhdlt.add(mhdlt1)
    }
}

createNhieuMhdd(from= ddh1, N0){
    nhieuMhdd= new ArrayList<>();

    for(N0 times){
        mh1= mh_Service.getRandom(1);

        mhdd1= mhdd_Faker.createFake(mh1);

        nhieuMhdd.add(mhdd1);
    }
}

In [ ]:
// vì MatHang có sẵn nên có thể lấy 1 loạt từ DB
// những cái khác phải create không thể lấy 1 loạt

createNhieuMhdlt(from= ch1, N0){
    nhieuMH= mh_Service.getRandom(N0);

    nhieuMhdlt= mhdlt_Faker.createFake(nhieuMH);
}

createNhieuMhdd(from= ddh1, N0){
    nhieuMH= mh_Service.getRandom(N0);

    nhieuMhdd= mhdd_Faker.createFake(nhieuMH);
}

In [ ]:
create all MatHang

create vpdd không có KH, không có CH 

for(all vpdd){
    thêm KH không có Ddh;

    thêm CH có Mhdlt
}

for(all KH){
    thêm Ddh có chứa Mhdd
}

for(all CH){
    thêm Mhdlt
}

//------- khởi tạo xong ------------


create KH , có 1 đơn hàng, số MH= 4->7

create 1 CH từ 1 thành phố ngẫu nhiên
(không create 2+ cửa hàng, vì không giống thực)

create 10 MatHang;



In [ ]:
Cannot invoke 

"com.example.demoKDLv1.Layer_Entity.MatHangDuocDat.MatHangDuocDat_Entity.MatHangDuocDat_Key.setDonDatHang_IdEmbedded(java.lang.Long)" 

because the return value of 

"com.example.demoKDLv1.Layer_Entity.MatHangDuocDat.MatHangDuocDat_Entity.

MatHangDuocDat.getMhddKey()" 

is null
-----------------------------------------------


/* kết luận= do chưa khởi tạo */

new MhddKey()// dùng cái này ở hàm khởi tạo của entity


In [ ]:
No default constructor for entity 

'com.example.demoKDLv1.Layer_Entity.MatHangDuocDat.MatHangDuocDat_Entity.MatHangDuocDat'

------------------------------------------------

/* kết luận= do thiếu hàm KT mặc định, hàm KT mặc định là hàm KT rỗng */

// https://stackoverflow.com/questions/4488716/java-default-constructor

In [ ]:
No default constructor for entity 'com.example.demoKDLv1.Layer_Entity.MatHang.MatHang_Entity.MatHang'

---------------------------------------
/* kết luận= mọi enity phải có hàm KT mặc định (là hàm KT rỗng) */

In [ ]:
Cannot invoke "java.util.Set.add(Object)" 

because "this.listMhdd" is null

------------------------------------------------

/* kết luận= phải khởi tạo this.listMhdd */
this.listMhdd= new Set<>()

biến được khai báo đi kèm với giá trị trong class java thì hệ thống sẽ đối xử với nó như thế nào

<details>

Khi một đối tượng Person được tạo ra mà không có đối số, thì biến name sẽ có giá trị mặc định là "John".

Nếu một đối tượng Person được tạo với tham số, thì giá trị name sẽ được thay thế bằng giá trị tham số.
</details>

In [ ]:
There was an unexpected error 
(type=Internal Server Error, status=500).

No message available

java.util.ConcurrentModificationException

---------------------------------------------------------
// https://stackoverflow.com/questions/8104692/how-to-avoid-java-util-concurrentmodificationexception-when-iterating-through-an

Hai lựa chọn:

<!> Tạo danh sách các giá trị bạn muốn xóa, thêm vào danh sách đó trong vòng lặp, sau đó gọi <!> originalList.removeAll(valuesToRemove) <!> ở cuối

Sử dụng remove()phương thức trên chính trình lặp. Lưu ý rằng điều này có nghĩa là bạn không thể sử dụng vòng lặp for nâng cao.


In [ ]:
// thay thế

public void appendListCuahang(List<CuaHang> listCh2){
    this.
}